In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [53]:
!ls /content/drive/My\ Drive/giraffe
!cp /content/drive/My\ Drive/giraffe/data.zip data.zip
!unzip data.zip > ziplog.txt


dataset_10000.hdf5  dataset_20000_128.hdf5  data.zip
dataset_1000.hdf5   dataset_50000.hdf5	    train
replace __MACOSX/._train_images? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train_images/train_image_png_707.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
import torch
from torch import nn
from torch.utils.data import random_split, Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
import torchvision
from torchvision.datasets import MNIST
import numpy as np
import h5py
from PIL import Image

In [0]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 3, 10, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [0]:
def load_dataset(path):
    with h5py.File(path, "r") as f:
        rgb = np.array(f["renders"])
        scenes = np.array(f["scenes"]).astype(np.uint8)
        # print(scenes[0])
        for index, img in enumerate(rgb):
            im = Image.fromarray(img)
            if index == 0:
              # im.show()
              im.save('example.png')
        return rgb, scenes

class RayTracingDataset(Dataset):

    def __init__(self, scenes, renders, transform=None):
        '''
        Args:
            scenes (Numpy array)
            renders (Numpy array)
            transform (callable, optional)
        '''
        self.scenes = scenes
        self.renders = renders
        self.transform = transform
    
    def __len__(self):
        return len(self.scenes)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'scene': self.scenes[idx],
                  'render': self.renders[idx]}

        return sample


In [0]:
split_factor = 0.8

# load the dataset
scenes, rgb = load_dataset(dataset_path)

transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

rt_dataset = RayTracingDataset(scenes=scenes, renders=rgb, transform=transform)
print("Dataset length is {}.".format(len(rt_dataset)))

dataset_split_size = [int(len(rt_dataset) * split_factor), 
                        len(rt_dataset) - int(len(rt_dataset) * split_factor)]



# split the dataset into train and test
train_dataset, test_dataset = random_split(rt_dataset, dataset_split_size)

In [71]:


# train_dataset = torchvision.datasets.STL10(root='.', split='train', transform=transform, download=True)
# test_dataset = torchvision.datasets.STL10(root='.', split='test', transform=transform, download=True)

b_size = 128
img_size = 128
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=b_size, shuffle=True, num_workers=4,
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=b_size, shuffle=False, num_workers=4
)


num_epochs = 500
learning_rate = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = autoencoder()
model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, 
    weight_decay=1e-5
)


Files already downloaded and verified
Files already downloaded and verified


In [72]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, img_size, img_size)
    return x

from torchvision.utils import save_image

losses = []
for epoch in range(num_epochs):
    for data, _ in train_loader:


        data = Variable(data).cuda()
        # data = data.view(data.size(0), -1)

        optimizer.zero_grad()

        output = model(data)


        loss = criterion(output, data)
        losses.append(loss)

        
        loss.backward()
        optimizer.step()
    
    
    if epoch % 10 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data))
        pic = to_img(output.cpu().data)
        save_image(pic, '/content/drive/My Drive/giraffe/train/{}.png'.format(str(epoch+1)))
import matplotlib.pyplot as plt

plt.plot(losses)
plt.show()
torch.save(model.state_dict(), '/content/drive/My Drive/giraffe/train/checkpoint.pth')

epoch [1/500], loss:0.2333
epoch [11/500], loss:0.1124
epoch [21/500], loss:0.0805
epoch [31/500], loss:0.0631
epoch [41/500], loss:0.1012
epoch [51/500], loss:0.0788
epoch [61/500], loss:0.0641
epoch [71/500], loss:0.0634
epoch [81/500], loss:0.0666
epoch [91/500], loss:0.0461
epoch [101/500], loss:0.0568
epoch [111/500], loss:0.0851
epoch [121/500], loss:0.0806
epoch [131/500], loss:0.1043
epoch [141/500], loss:0.0826
epoch [151/500], loss:0.0649
epoch [161/500], loss:0.0777
epoch [171/500], loss:0.0709
epoch [181/500], loss:0.0652
epoch [191/500], loss:0.0816
epoch [201/500], loss:0.0718
epoch [211/500], loss:0.0557
epoch [221/500], loss:0.0746
epoch [231/500], loss:0.0745
epoch [241/500], loss:0.0595
epoch [251/500], loss:0.0542
epoch [261/500], loss:0.0635
epoch [271/500], loss:0.0536
epoch [281/500], loss:0.0809
epoch [291/500], loss:0.0603
epoch [301/500], loss:0.0879
epoch [311/500], loss:0.0630
epoch [321/500], loss:0.0558
epoch [331/500], loss:0.0602
epoch [341/500], loss:0.0

KeyboardInterrupt: ignored